# load_ecology_bottle

This notebook extracts, transforms, and loads Ecology lab/bottle data from their Marine Water study. This data is not available for direct download and instead must be retrieved through a public records request.

From the data I have reviewed, the format is not entirely consistent in how the spreadsheets are laid out but there is enough of a common pattern that this notebook can handle at least a few cases. Your mileage may vary, unfortunately.

The format I have for this data is as .zip files that contain one or more Excel workbooks. If yours are the same format, just place the .zip files in the directory `data/ecology/bottle` and this notebook will find them.

This notebook only imports data that has fully passed QA: the QA column for each observation must contain "2_0_3". See Ecology's documentation, which should be included with the files, to understand what this means.

In [13]:
ecology_bottle_files = "data/ecology/bottle/*.zip"

import glob
import zipfile
import re
import multiprocessing as mp
import pandas as pd
from sqlalchemy import text
from openpyxl import load_workbook
import numpy as np
import db

Construct a DataFrame that maps between the database parameter IDs, the corresponding Ecology parameter column names, and a multiplier for unit conversions.

In [2]:
ecology_column_names = pd.DataFrame({
    "raw_name": ["Chla_Lab","Chla_Lab (ug/L)","NH4_Lab","NH4(uM)D",
                 "NO3_Lab","NO3(uM)D","NO2_Lab","NO2(uM)D","PO4_Lab",
                 "PO4(uM)D","SiOH4_Lab","SiOH4(uM)D"],
    "key_name": ["chla","chla","nh4","nh4","no3","no3","no2","no2",
                 "po4","po4","sioh4","sioh4"],
    "conv": [1,1,1,1,1,1,1,1,1,1,1,1]
})
ecology_column_names

,raw_name,key_name,conv
0,Chla_Lab,chla,1
1,Chla_Lab (ug/L),chla,1
2,NH4_Lab,nh4,1
3,NH4(uM)D,nh4,1
4,NO3_Lab,no3,1
5,NO3(uM)D,no3,1
6,NO2_Lab,no2,1
7,NO2(uM)D,no2,1
8,PO4_Lab,po4,1
9,PO4(uM)D,po4,1


Fetch the Ecology source ID, which was created in [load_ecology_ctd](load_ecology_ctd.ipynb)

In [3]:
engine = db.connect()
# Refresh the sources so we can fetch the primary key
df = pd.read_sql_table("sources", con=engine, schema='obsdata', index_col='id')
ecology_source_id = df.loc[(df['agency'] == "WA Ecology") & (df['study'] == "MarineWater")].index[0]
print(ecology_source_id)

1


Process all the spreadsheets found in .zip files in the target directory.

Individual Excel files can be vastly different sizes, so it makes sense to use a producer/consumer multiprocessing model here with a Queue. Unfortunately we need to apply deduplication before commiting to the DB, so a Manager is used with a server process to gather all the result data. A lock is shared between the workers since the column list update process is not thread-safe.

In [15]:
spreadsheet_re = re.compile(".*\\.xlsx")

processed_data = []

def build_df_from_zip(zip_file, data_file):
    with zipfile.ZipFile(zip_file) as zf:
        with zf.open(data_file) as data_f:
            # Some spreadsheets Ecology sends have multiple worksheets we need to parse
            wb = load_workbook(data_f)
            for sheet in wb.sheetnames[4:-1]:
                # Ignore CTD data as we already have this from EIM
                if "CTD" in sheet:
                    continue
                yield pd.read_excel(data_f, sheet_name=sheet, parse_dates=[2], na_values=-9999)
                
def worker(inq, lock, outs):
    thread_con = db.connect()
    while True:
        zip_file, data_file = inq.get()
        if zip_file is None:
            break
        for df in build_df_from_zip(zip_file, data_file):
            # Ecology bottle data does not have full timestamps, only the date. To
            # remedy this, go off the knowledge that bottles were filled on the
            # upcast. Find the corresponding downcast for each sample by matching
            # date/station and look for the latest time.
            stations = df["Station"].drop_duplicates().to_list()
            stations_dict = {}
            stations_strs = []
            for n,s in enumerate(stations):
                stations_strs.append(f':st{n}')
                stations_dict[f'st{n}'] = s
            # See https://stackoverflow.com/a/53839782
            stmt = text("SELECT location_id, cast_id, MAX(datetime) AS datetime "
                    f"FROM obsdata.observations WHERE location_id IN ({', '.join(stations_strs)}) "
                    "GROUP BY location_id, cast_id"
                   )
            downcasts = pd.read_sql(stmt, con=engine, params=stations_dict)
            downcasts['date'] = pd.to_datetime(
                pd.to_datetime(downcasts['datetime'], utc=True).dt.date)
            df = df.merge(downcasts, how='left', left_on=['Station','Date'], right_on=['location_id','date'])
            # Prevent unique constraint violations with the CTD data by adding a
            # minute to the time
            df['datetime'] += pd.Timedelta(1, 'min')
            # Fallback if we don't have a matching downcast for particular data:
            # just use the raw date
            df['datetime'].fillna(df['Date'], inplace=True)

            # Drop anything with missing values
            df.dropna(subset='Depth_Matching', inplace=True)

            # Each spreadsheet has different column names depending on the type of
            # data that was given. Go off a lookup table to match column name with
            # parameter ID, and perform any necessary unit conversions
            for i,row in ecology_column_names.iterrows():
                param = row['raw_name']
                if param not in df.columns:
                    continue
                # For each found column, replace values with NaN if the first character
                # of the corresponding QC column (the next column over) is not 2_0_3
                qc = df[df.columns[df.columns.get_loc(param)+1]]
                df.loc[qc != '2_0_3', param] = np.nan

                # For each data column, make a view that drops the NaNs, then append
                # that view's location_id, datetime, depth, and column value to
                # outs. Create a filled parameter_id that corresponds to the
                # data column and append it as well.
                view = df.dropna(subset=param)

                with lock:
                    outs['location_id'].extend(view['Station'].to_list())
                    outs['datetime'].extend(view['datetime'].to_list())
                    outs['depth'].extend(view['Depth_Matching'].to_list())
                    outs['value'].extend(view[param].to_list())
                    outs['parameter_id'].extend([row['key_name']] * len(view))

# For reference/inspiration, see:
# https://docs.python.org/3/library/multiprocessing.html#sharing-state-between-processes
# https://stackoverflow.com/a/43079667
inq = mp.Queue()
lock = mp.Lock()
with mp.Manager() as manager:
    outs = manager.dict()
    for k in ['location_id','datetime','depth','value','parameter_id']:
        outs[k] = manager.list()
    with mp.Pool(initializer=worker, initargs=(inq, lock, outs)) as pool:
        for f in glob.glob(ecology_bottle_files):
            with zipfile.ZipFile(f) as zf:
                files = zf.namelist()
                # Find the files in this zip that match the data filename pattern
                # See https://stackoverflow.com/a/19502692
                data_files = filter(lambda s: s if spreadsheet_re.match(s) else None, files)
                for data_file in data_files:
                    print(data_file)
                    inq.put((f, data_file))
        # We're done, signal to the workers to terminate
        for _ in range(pool._processes):
            inq.put((None, None))
        pool.close()
        pool.join()
    # Copy the shared memory locally
    processed_data = {}
    for k in outs:
        processed_data[k] = list(outs[k])
    processed_data = pd.DataFrame(processed_data)
    processed_data['source_id'] = ecology_source_id

# There may be overlap in data between the workbooks, so drop any full duplicates
processed_data.drop_duplicates(inplace=True)
# There are occasional duplicates in the dataset with non-matching values, probably
# due to depth mismatches. To prevent mistakes just drop all of them
processed_data.drop_duplicates(subset=['datetime','depth','parameter_id','location_id'],
                               keep=False, inplace=True)

processed_data.to_sql('observations', con=engine, schema='obsdata', index=False, if_exists='append')
print(f'{len(processed_data)} Total rows found')
processed_data.head()

/tmp/ipykernel_1391364/1490424298.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  yield pd.read_excel(data_f, sheet_name=sheet, parse_dates=[2], na_values=-9999)


KingCounty_Apr2018DataRequest/KingCounty_JEMSChlaData_Aug2018.xlsx
KingCounty_Apr2018DataRequest/KingCounty_JEMSData_Apr2018.xlsx
KingCounty_Apr2018DataRequest/KingCounty_MainBasinChla&NutData_Jul2018.xlsx
KingCounty_Apr2018DataRequest/KingCounty_MainBasinData_Apr2018.xlsx
KingCounty_Apr2018DataRequest/KingCounty_QMHarborData_Apr2018.xlsx


/tmp/ipykernel_1391364/1490424298.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2001-02-26 00:00:00', '2001-03-20 00:00:00', '2001-03-20 00:00:00',
 '2001-04-26 00:00:00', '2001-05-21 00:00:00', '2001-05-21 00:00:00',
 '2001-06-19 00:00:00', '2001-06-19 00:00:00', '2001-07-18 00:00:00',
 '2001-07-18 00:00:00', '2001-08-20 00:00:00', '2001-08-20 00:00:00',
 '2001-11-27 00:00:00', '2001-12-28 00:00:00', '2004-02-19 00:00:00',
 '2004-02-19 00:00:00', '2004-03-10 00:00:00', '2004-03-10 00:00:00',
 '2004-04-06 00:00:00', '2004-04-06 00:00:00', '2004-05-03 00:00:00',
 '2004-05-03 00:00:00', '2004-06-01 00:00:00', '2004-06-01 00:00:00',
 '2004-08-10 00:00:00', '2004-08-10 00:00:00', '2004-09-21 00:00:00',
 '2004-09-21 00:00:00', '2004-10-26 00:00:00', '2004-10-26 00:00:00',
 '2010-01-14 00:00:00', '2010-01-14 00:00:00', '2010-02-10 00:00:00',
 '2010-02-10 00:00:00', '2010-03-02 00:00:00', '2010-04

13157 Total rows found


,location_id,datetime,depth,value,parameter_id,source_id
0,QMH002,2001-02-26 17:43:43+00:00,0.5,0.12,nh4,1
1,QMH002,2001-03-20 23:53:35+00:00,0.5,0.95,nh4,1
2,QMH002,2001-03-20 23:53:35+00:00,10.0,1.18,nh4,1
3,QMH002,2001-04-26 00:00:00,0.5,0.21,nh4,1
4,QMH002,2001-05-21 23:41:22+00:00,0.5,0.11,nh4,1
